# Analyzing ICWSM Ethics Statements
### Author: Campbell Lund
### 6/22/2023
This notebook contains the preprocessing and NLP analysis for the 2022 and 2023 ethics statements published by the ICWSM Conference.

### Table of contents:
1. [Preprocessing](#sec1)
2. [Word Frequency](#sec2)
3. [Sentence Analysis](#sec3)
4. [Topic Modeling](#sec4)
5. [LLM Evaluation](#sec5)

## 1. Preprocessing <a name="sec1"></a>

In [120]:
import pandas as pd

In [121]:
# load data
df22 = pd.read_csv('data/ICWSM2022EthicsStatements.csv')
df23 = pd.read_csv('data/ICWSM2023EthicsStatements.csv')

In [122]:
len(df23)

108

In [123]:
df22_length = len(df22)
df23_length = len(df23)

# remove NaN values
df22 = df22.dropna(inplace=False)
df23 = df23.dropna(inplace=False)

# calculate the number of papers without ethics statements
num_without_statements22 = df22_length - len(df22)
num_without_statements23 = df23_length - len(df23)

print('Number of papers without ethics statements for 2022: ',num_without_statements22)
print('Number of papers without ethics statements for 2023: ',num_without_statements23)

# rework to save the papers wo statements in a seperate df: why is dropna not defaulting to this???

Number of papers without ethics statements for 2022:  80
Number of papers without ethics statements for 2023:  11


In [124]:
# add a column to the df for ethics statement word count and assign a unique paper ID
df22['Word Count'] = 0
df23['Word Count'] = 0
df22['ID'] = 0
df23['ID'] = 0
counter = 0

for i, row in df22.iterrows():
    text = df22.loc[i,'Ethics']
    words = text.split()
    df22.loc[i,'Word Count'] = len(words)
    df22.loc[i,'ID'] += counter
    counter += 1

counter = 0

for i, row in df23.iterrows():
    text = df23.loc[i,'Ethics']
    words = text.split()
    df23.loc[i,'Word Count'] = len(words)
    df23.loc[i,'ID'] = counter
    counter += 1


In [125]:
df23

,Paper Title,Authors,Section Title,Ethics,Word Count,ID
0,How Do US Congress Members Advertise Climate Change: An Analysis of Ads Run on Meta’s Platforms,"Laurenz Aisenpreis, Gustav Gyrst, Vedran Sekara",Ethical Statement,"The data in this paper is derived from the Meta Ad Library. It contains publicly accessible ads run on Meta platforms by US politicians. Working with social media data carries risks of privacy issues and the right to be forgotten. However, our data analysis is limited to aggregated data presentations and only concerns ads published by public figures.",58,0
1,The Pursuit of Peer Support for Opioid Use Recovery on Reddit,"Duilio Balsamo, Paolo Bajardi, Gianmarco De Francisci Morales, Corrado Monti, Rossano Schifanella",Ethical Statement,"This work follows the guidelines and the ethical considerations by Eysenbach and Till (2001); Moreno et al. (2013); Ramırez-Cifuentes et al. (2020). All the results provide aggregated estimates and do not include any information on individuals. The users in our study were fully aware of the public nature and free accessibility of the content they posted since the subreddits are of public domain, are not password-protected, and have thousands of active subscribers. Reddit’s pseudonymous accounts make the retrieval of the true identity of users unlikely. Nevertheless, as a further privacy measure, the authors’ names were anonymized before using the data for analysis. Therefore, our research did not require informed consent.",110,1
2,Exposure to Marginally Abusive Content on Twitter,"Jack Bandy, Tomo Lazovich",Ethics Statement,"This study intersects with a number of topics related to the ethics of algorithmic platforms. For example, our analysis requires collecting data about which Tweets users view while using Twitter, and also requires human annotators to review and rate potentially abusive content. Overall, we agree with researchers who view this type of work as necessary for understanding and protecting democratic discourse (Fiske 2022), especially in terms of standard risk-benefit frameworks. Still, it is important to note different measures taken to address potential risks. While the holdback experiment is necessary, it is not ideal for many users to be excluded from algorithmic timeline features. Twitter has thus worked to provide more users access to algorithmic timelines while maintaining statistical robustness in the holdback experiment. As of 2020, the experiment included over 2 million active accounts in the reverse-chronological timeline group (Husz ́ar et al. 2022), but when this analysis was conducted in 2021, that number had been reduced to 630k. This study was not subject to an academic IRB process, however, it went through standard legal and privacy review processes at Twitter. Finally, the data used in this paper was fully anonymized for publication, following standard ethical procedures. We do not include any results that might disclose the identity of any account in the datasets.",214,2
3,Finding Qs: Profiling QAnon Supporters on Parler,"Dominik Bär, Nicolas Pröllochs, Stefan Feuerriegel",Ethics Statement,"This research did not involve interventions with human subjects, and, thus, no approval from the Institutional Review Board was required by the author institutions. All analyses are based on publicly available data and we do not make any attempt to track users across different platforms. We neither de-anonymize nor de-identify their accounts. Furthermore all analyses conform with national laws. To respect privacy, we explicitly do not publish usernames in our paper (except for celebrity profiles) and only report aggregate results.",80,3
4,Predicting Future Location Categories of Users in a Large Social Platform,"Raiyan Abdul Baten, Yozen Liu, Heinrich Peters, Francesco Barbieri, Neil Shah, Leonardo Neves, Maarten W. Bos",Ethics Statement,"Any experiment dealing with data as sensitive as ours (e.g.,location) needs to operate ethically and securely. Our appro

In [126]:
# calculate average word count
print('Average word count 2022: ', int(df22['Word Count'].mean()))
print('Average word count 2023: ',int(df23['Word Count'].mean()))

Average word count 2022:  184
Average word count 2023:  196


In [127]:
#pd.set_option('display.max_colwidth', None)
df23.sort_values(by=['Word Count'],ascending=False)

Paper Title   
42   Popular Support for Balancing Equity and Efficiency in Resource Allocation: A Case Study in Online Advertising to Increase Welfare Program Awareness  \
5                                              Followback Clusters, Satellite Audiences, and Bridge Nodes: Coengagement Networks for the 2020 US Election   
18                                                                                                                      Misleading Repurposing on Twitter   
44                                                                     Large-Scale Demographic Inference of Social Media Users in a Low-Resource Scenario   
16                                                                     We Are in This Together: Quantifying Community Subjective Wellbeing and Resilience   
..                                                                                                                                                    ...   
57                                                                                        The Chance of Winning Election Impacts on Social Media Strategy   
36                                                 Online Emotions during the Storming of the U.S. Capitol: Evidence from the Social Media Network Parler   
70                                                                                                       Social Influence-Maximizing Group Recommendation   
84                                                                A Multi-Platform Collection of Social Media Posts about the 2022 U.S. Midterm Elections   
102                                                                                                      The Amplification Paradox in Recommender Systems   

                                                                                                                                                                                                                                                              Authors   
42                                                                                                                                                                                                         Allison Koenecke, Eric Giannella, Robb Willer, Sharad Goel  \
5                                                                                                                                                                             Andrew Beers, Joseph S. Schafer, Ian Kennedy, Morgan Wack, Emma S. Spiro, Kate Starbird   
18                                                                                                                                                                                                                     Tuğrulcan Elmas, Rebekah Overdorf, Karl Aberer   
44                                                                                                                                               Karim Lasri, Manuel Tonneau, Haaya Naushan, Niyati Malhotra, Ibrahim Farouq, Víctor Orozco-Olvera, Samuel Fraiberger   
16                                                                                                                                                                                                            MeiXing Dong, Ruixuan Sun, Laura Biester, Rada Mihalcea   
..                                                                                                                                                                                                                                                                ...   
57                                                                                                                                                                                Taichi Murayama, Akira Matsui, Kunihiro Miyazaki, Yasuko Matsubara, Yasushi Sakurai   
36                                                                                                                                       

In [128]:
import re

In [129]:
def remove_citations(df,patterns):
    citation_df = pd.DataFrame({'ID':[],'Citations':[]})
    counter = 0
    for i, row in df.iterrows():
        text = df.loc[i,'Ethics']
    
        for j, p in enumerate(patterns):
            citations = re.findall(p,text)
    
            # if citations are found, remove them from the text and add to a seperate df
            if (len(citations) != 0):
                text = re.sub(p,'',text)
                for c in citations:
                    ID = str(i)+str(counter)
                    new_row = {'ID': ID, 'Citations': c}
                    citation_df.loc[len(citation_df)] = new_row
                    counter += 1
                
                # replace previous text with cleaned version
                df.loc[i,'Ethics'] = text
        
        # reset the counter before each text
        counter = 0
    return (citation_df, df)

In [130]:
# remove the following regular expressions for citations from the df and store them in a seperate df with unique ID (ID = paper ID + citation index)
patterns = [r'\(\d{4}\)', r'\(\d+\)', r'\([A-Za-z]+\s\d{4}\)', r'\([A-Za-z]+\d{4}\)', r'\b\d+\b', r'\set\sal\.']

citation_data22 = remove_citations(df22,patterns)
citation_df22 = citation_data22[0]
df22 = citation_data22[1]

citation_data23 = remove_citations(df23,patterns)
citation_df23 = citation_data23[0]
df23 = citation_data23[1]

In [131]:
citation_df23

,ID,Citations
0,10,(2001)
1,11,(2013)
2,12,(2020)
3,13,et al.
4,14,et al.
...,...,...
196,971,22
197,972,129
198,980,2019
199,1030,1000


In [132]:
df23

,Paper Title,Authors,Section Title,Ethics,Word Count,ID
0,How Do US Congress Members Advertise Climate Change: An Analysis of Ads Run on Meta’s Platforms,"Laurenz Aisenpreis, Gustav Gyrst, Vedran Sekara",Ethical Statement,"The data in this paper is derived from the Meta Ad Library. It contains publicly accessible ads run on Meta platforms by US politicians. Working with social media data carries risks of privacy issues and the right to be forgotten. However, our data analysis is limited to aggregated data presentations and only concerns ads published by public figures.",58,0
1,The Pursuit of Peer Support for Opioid Use Recovery on Reddit,"Duilio Balsamo, Paolo Bajardi, Gianmarco De Francisci Morales, Corrado Monti, Rossano Schifanella",Ethical Statement,"This work follows the guidelines and the ethical considerations by Eysenbach and Till ; Moreno ; Ramırez-Cifuentes . All the results provide aggregated estimates and do not include any information on individuals. The users in our study were fully aware of the public nature and free accessibility of the content they posted since the subreddits are of public domain, are not password-protected, and have thousands of active subscribers. Reddit’s pseudonymous accounts make the retrieval of the true identity of users unlikely. Nevertheless, as a further privacy measure, the authors’ names were anonymized before using the data for analysis. Therefore, our research did not require informed consent.",110,1
2,Exposure to Marginally Abusive Content on Twitter,"Jack Bandy, Tomo Lazovich",Ethics Statement,"This study intersects with a number of topics related to the ethics of algorithmic platforms. For example, our analysis requires collecting data about which Tweets users view while using Twitter, and also requires human annotators to review and rate potentially abusive content. Overall, we agree with researchers who view this type of work as necessary for understanding and protecting democratic discourse , especially in terms of standard risk-benefit frameworks. Still, it is important to note different measures taken to address potential risks. While the holdback experiment is necessary, it is not ideal for many users to be excluded from algorithmic timeline features. Twitter has thus worked to provide more users access to algorithmic timelines while maintaining statistical robustness in the holdback experiment. As of , the experiment included over million active accounts in the reverse-chronological timeline group (Husz ́ar ), but when this analysis was conducted in , that number had been reduced to 630k. This study was not subject to an academic IRB process, however, it went through standard legal and privacy review processes at Twitter. Finally, the data used in this paper was fully anonymized for publication, following standard ethical procedures. We do not include any results that might disclose the identity of any account in the datasets.",214,2
3,Finding Qs: Profiling QAnon Supporters on Parler,"Dominik Bär, Nicolas Pröllochs, Stefan Feuerriegel",Ethics Statement,"This research did not involve interventions with human subjects, and, thus, no approval from the Institutional Review Board was required by the author institutions. All analyses are based on publicly available data and we do not make any attempt to track users across different platforms. We neither de-anonymize nor de-identify their accounts. Furthermore all analyses conform with national laws. To respect privacy, we explicitly do not publish usernames in our paper (except for celebrity profiles) and only report aggregate results.",80,3
4,Predicting Future Location Categories of Users in a Large Social Platform,"Raiyan Abdul Baten, Yozen Liu, Heinrich Peters, Francesco Barbieri, Neil Shah, Leonardo Neves, Maarten W. Bos",Ethics Statement,"Any experiment dealing with data as sensitive as ours (e.g.,location) needs to operate ethically and securely. Our approach actively aims to minimize risks of misuse and intrusion by av

In [133]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import string

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\campb\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\campb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [134]:
stopwords = stopwords.words('english')
tokenizer = word_tokenize

In [135]:
def clean_text_words(text):
    tokens = word_tokenize(text)
    
    # make text lowercase
    lowercase_tokens = [w.lower() for w in tokens]
    
    # remove punctuation
    remove_punct_tokens = [w for w in lowercase_tokens if w not in string.punctuation]
    
    # remove stopwords
    clean_tokens = [w for w in remove_punct_tokens if w not in stopwords]
    return clean_tokens

In [136]:
def clean_text_sents(text):
    tokens = sent_tokenize(text)
    
    # make text lowercase
    lowercase_tokens = [w.lower() for w in tokens]
    
    return lowercase_tokens

In [137]:
tokenized_words22 = df22.copy()
tokenized_words23 = df23.copy()

tokenized_sents22 = df22.copy()
tokenized_sents23 = df23.copy()

In [138]:
# clean and tokenize each word in the ethics statement
for col in tokenized_words22.columns[:-2]:
    tokenized_words22[col] = tokenized_words22[col].apply(clean_text_words)

for col in tokenized_words23.columns[:-2]:
    tokenized_words23[col] = tokenized_words23[col].apply(clean_text_words)

In [139]:
# clean and tokenize each sentence in the ethics statement
for col in tokenized_sents22.columns[:-2]:
    tokenized_sents22[col] = tokenized_sents22[col].apply(clean_text_sents)

for col in tokenized_sents23.columns[:-2]:
    tokenized_sents23[col] = tokenized_sents23[col].apply(clean_text_sents)

## 2. Word Frequency <a name="sec2"></a>

In [140]:
from nltk import FreqDist

In [141]:
freq_df22 = pd.DataFrame({'ID':[],'Frequent Words':[]})
freq_df23 = pd.DataFrame({'ID':[],'Frequent Words':[]})

In [142]:
# propagate freq_df with the 5 most frequently used words for each paper
for i, row in tokenized_words22.iterrows():
    freq_dist = FreqDist(tokenized_words22.loc[i,'Ethics'])
    most_common_words = freq_dist.most_common(5)
    
    freq_df22.loc[i,'ID'] = tokenized_words22.loc[i,'ID']
    freq_df22.loc[i,'Frequent Words'] = [most_common_words]
    
for i, row in tokenized_words23.iterrows():
    freq_dist = FreqDist(tokenized_words23.loc[i,'Ethics'])
    most_common_words = freq_dist.most_common(5)
    
    freq_df23.loc[i,'ID'] = tokenized_words23.loc[i,'ID']
    freq_df23.loc[i,'Frequent Words'] = [most_common_words]

In [143]:
freq_df23

,ID,Frequent Words
0,0.0,"[(data, 4), (meta, 2), (ads, 2), (paper, 1), (derived, 1)]"
1,1.0,"[[(users, 2), (public, 2), (’, 2), (work, 1), (follows, 1)]]"
2,2.0,"[[(algorithmic, 3), (users, 3), (twitter, 3), (standard, 3), (experiment, 3)]]"
3,3.0,"[[(analyses, 2), (research, 1), (involve, 1), (interventions, 1), (human, 1)]]"
4,4.0,"[[(data, 3), (sensitive, 2), (thus, 2), (snapchat, 2), (’, 2)]]"
...,...,...
101,92.0,"[[(twitter, 4), (hate, 3), (api, 2), (data, 1), (collected, 1)]]"
102,93.0,"[[(foresee, 1), (negative, 1), (societal, 1), (impact, 1), (coming, 1)]]"
103,94.0,"[[(risk, 3), (data, 2), (facebook, 2), (’, 2), (given, 2)]]"
105,95.0,"[[(data, 4), (sms, 3), (implications, 2), (thus, 2), (social, 2)]]"


## 3. Sentence Analysis <a name="sec3"></a>

In [144]:
tokenized_sents22

,Paper Title,Authors,Section Title,Ethics,Word Count,ID
0,"[leaders or followers?, a temporal analysis of tweets from ira trolls]","[siva k. balasubramanian, mustafa bilgic, aron culotta, libby hemphill, anita nikolich, matthew a. shapiro]",[ethical statement],"[the data in this paper is derived from publicly accessible user-generated content online., while our focus is on aggregate trending keywords and not individual user characteristics, such data carry risks for issues of privacy and “right-to-be-forgotten.” to mitigate these issues and comply with terms of service, we will release only tweet ids for the data used in this study.]",59,0
1,[community under surveillance: impacts of marginalization on an online labor forum],"[hanna barakat, elissa m. redmiles]",[ethics statement],"[our work was approved by our institution’s ethics review board., despite this approval, tension exists within the literature regarding the ethics of analyzing ‘public’ forum data and why, when, and how users may perceive public forum discussions as private (e.g., proferes et al., ; vi-tak, shilton, and ashktorab ; eysenbach and till ; razi, badillo-urquiola, and wisniewski )., in this work, we follow guidance from cook, ayers, and horsch and dym and fiesler on preserving the platform’s anonymity and draw on the established framework from eysenbach and till to assess the ethics of our work., eysenbach and till suggest three central aspects to evaluating the ‘private’ or ‘public’ nature of online sources., first is an assent of access to the forum: is registration necessary to view content or post?, this specific forum is an ‘open’ forum and the social media platform does not require registration., second, forum size; this forum had approximately , members at the time of data collection, which is between a medium- and large-sized forum for this platform., third, eysenbach and till recommend assessing how members perceive the forum., it is difficult to gauge whether members perceive the forum as public or private., however, posts and comments often allude to the public nature of the content posted., for example, members warned each other not to post specific platform names in case law enforcement was monitoring the page and frequently suggested moving conversations out of public view to direct messages., while our assessment along these three criteria suggests that forum members are aware of the public nature of the platform, to protect participants we omit usernames user names (cook, ayers, and horsch ; dym and fiesler2020), anonymize the platform and forum name, alter or paraphrase all quotes so they cannot be reverse-searched, and the names of social media sites, messaging applications,and other tools are intentionally removed from this paper to avoid any harmful repercussions to sex workers using those tools (costanza-chock ).]",327,1
3,"[linguistic characterization of divisive topics online: case studies on contentiousness in abortion, climate change, and gun control]","[jacob beel, tong xiang, sandeep soni, diyi yang]",[ethics statement],"[this research study has been approved by the institutional review board (irb) at the researchers’ institution., in this work, we leverage no information that was not publicly available at the time of data collection, leveraging the public post histories of the users who participated in the conversations which we studied., as such, user private information is not disclosed, and none of the posts which were used to compute the gender and location features have been saved or need to be saved during the course of this computation.]",87,2
5,"[to recommend or not?, a model-based comparison of item-matching processes]","[serina chang, johan ugander]",[broader impact],"[our work is primarily motivated by broader impacts: our goal is to develop a principled framework through which we can meaningfully assess the impacts of recommender systems on society., recommender systems are ubiquitous on the web and social media pla

In [145]:
sentence_df22 = pd.DataFrame({'ID':[],'Sentence':[]})
sentence_df23 = pd.DataFrame({'ID':[],'Sentence':[]})

In [146]:
# make a new df of sentences with unique IDs (ID = paper ID + sentence index)
for i, row in tokenized_sents22.iterrows():
    sents = tokenized_sents22.loc[i,'Ethics']
    paper_ID = tokenized_sents22.loc[i,'ID']
    ID = i
    for j, s in enumerate(sents):
        ID = str(i)+str(j)
        new_row = {'ID': ID, 'Sentence': s}
        sentence_df22.loc[len(sentence_df22)] = new_row
        
for i, row in tokenized_sents23.iterrows():
    sents = tokenized_sents23.loc[i,'Ethics']
    paper_ID = tokenized_sents23.loc[i,'ID']
    ID = i
    for j, s in enumerate(sents):
        ID = str(i)+str(j)
        new_row = {'ID': ID, 'Sentence': s}
        sentence_df23.loc[len(sentence_df23)] = new_row

In [147]:
sentence_df23

,ID,Sentence
0,00,the data in this paper is derived from the meta ad library.
1,01,it contains publicly accessible ads run on meta platforms by us politicians.
2,02,working with social media data carries risks of privacy issues and the right to be forgotten.
3,03,"however, our data analysis is limited to aggregated data presentations and only concerns ads published by public figures."
4,10,this work follows the guidelines and the ethical considerations by eysenbach and till ; moreno ; ramırez-cifuentes .
...,...,...
826,1072,"our only variables extracted from the twitterdata were tweet ids, timestamps of when the tweets were created, and the impression count, which is part of the public metric variable."
827,1073,"no tweet texts, account profile information, or other information that could identify individuals or groups (pii) were analyzed."
828,1074,reproducibility: all data from the analyses of this article are available online (www.pfeffer.at/data/halflife).
829,1075,"the data includes all tweet ids, tweet creation time, and for each collection iteration for every tweet, its collection time, and the number of views."


## 4. Topic Modeling <a name="sec4"></a>

In [148]:
import gensim
from gensim import corpora, models
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from gensim.models import LdaModel
from gensim.corpora import Dictionary

In [189]:
# takes the tokenized ethics statements and the number of topics as input and returns a lda model
def make_lda_model(tokenized_words,num_topics):
    dictionary = corpora.Dictionary(tokenized_words)
    corpus = [dictionary.doc2bow(doc) for doc in tokenized_words]
    lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)
    
    return lda_model

In [192]:
lda_model22 = make_lda_model(tokenized_words22['Ethics'],5)
lda_model23 = make_lda_model(tokenized_words23['Ethics'],5)

In [193]:
topics22 = lda_model22.show_topics(num_topics=num_topics, num_words=10)
for topic_id, topic_words in topics22:
    print(f"Topic ID: {topic_id}")
    print(f"Keywords: {topic_words}")
    print()

Topic ID: 2
Keywords: 0.022*"data" + 0.014*"dataset" + 0.011*"’" + 0.009*"users" + 0.008*"social" + 0.007*"used" + 0.007*"”" + 0.007*"“" + 0.006*"use" + 0.006*"twitter"

Topic ID: 0
Keywords: 0.013*"dataset" + 0.012*"data" + 0.010*"paper" + 0.008*"user" + 0.007*"use" + 0.006*"hate" + 0.006*"outlets" + 0.006*"articles" + 0.006*"available" + 0.006*"made"

Topic ID: 4
Keywords: 0.011*"subreddit" + 0.011*"users" + 0.010*"data" + 0.009*"forum" + 0.009*"links" + 0.008*"’" + 0.008*"work" + 0.008*"may" + 0.008*"public" + 0.006*"could"



In [194]:
topics23 = lda_model23.show_topics(num_topics=num_topics, num_words=10)
for topic_id, topic_words in topics23:
    print(f"Topic ID: {topic_id}")
    print(f"Keywords: {topic_words}")
    print()

Topic ID: 4
Keywords: 0.023*"data" + 0.012*"’" + 0.011*"twitter" + 0.011*"users" + 0.009*"dataset" + 0.009*"work" + 0.007*"information" + 0.007*"study" + 0.007*"tweet" + 0.006*"social"

Topic ID: 2
Keywords: 0.020*"data" + 0.012*"work" + 0.009*"media" + 0.008*"social" + 0.007*"may" + 0.006*"users" + 0.005*"research" + 0.005*"ethical" + 0.005*"used" + 0.005*"analysis"

Topic ID: 0
Keywords: 0.009*"information" + 0.007*"users" + 0.007*"study" + 0.005*"content" + 0.005*"dataset" + 0.004*"may" + 0.004*"also" + 0.004*"social" + 0.004*"health" + 0.004*"findings"



In [195]:
vis_data22 = gensimvis.prepare(lda_model22, corpus22, dictionary22)
vis_data23 = gensimvis.prepare(lda_model23, corpus23, dictionary23)

In [196]:
pyLDAvis.display(vis_data22)

In [158]:
pyLDAvis.display(vis_data23)

In [197]:
# topics of the 10 longest statements
long_statements22 = tokenized_words22.sort_values(by=['Word Count'],ascending=False)[:10]
long_lda_model22 = make_lda_model(long_statements22['Ethics'],3)

long_statements23 = tokenized_words23.sort_values(by=['Word Count'],ascending=False)[:10]
long_lda_model23 = make_lda_model(long_statements23['Ethics'],3)

In [198]:
long_topics22 = long_lda_model22.show_topics(num_topics=num_topics, num_words=10)
for topic_id, topic_words in long_topics22:
    print(f"Topic ID: {topic_id}")
    print(f"Keywords: {topic_words}")
    print()

Topic ID: 0
Keywords: 0.012*"dataset" + 0.010*"data" + 0.010*"links" + 0.010*"subreddit" + 0.008*"community" + 0.008*"users" + 0.007*"content" + 0.006*"use" + 0.006*"’" + 0.006*"may"

Topic ID: 1
Keywords: 0.022*"dataset" + 0.013*"’" + 0.009*"social" + 0.009*"data" + 0.008*"fields" + 0.008*"“" + 0.008*"”" + 0.007*"metadata" + 0.007*"media" + 0.007*"may"

Topic ID: 2
Keywords: 0.018*"data" + 0.013*"forum" + 0.011*"public" + 0.010*"recommender" + 0.010*"systems" + 0.010*"set" + 0.009*"’" + 0.008*"social" + 0.007*"platform" + 0.007*"nature"



In [199]:
long_topics23 = long_lda_model23.show_topics(num_topics=num_topics, num_words=10)
for topic_id, topic_words in long_topics23:
    print(f"Topic ID: {topic_id}")
    print(f"Keywords: {topic_words}")
    print()

Topic ID: 0
Keywords: 0.010*"social" + 0.009*"work" + 0.008*"data" + 0.008*"may" + 0.008*"communities" + 0.007*"media" + 0.007*"users" + 0.006*"wellbeing" + 0.006*"pandemic" + 0.004*"however"

Topic ID: 1
Keywords: 0.022*"data" + 0.013*"users" + 0.011*"dataset" + 0.008*"accounts" + 0.007*"twitter" + 0.007*"information" + 0.006*"content" + 0.006*"social" + 0.006*"potential" + 0.006*"media"

Topic ID: 2
Keywords: 0.008*"information" + 0.007*"research" + 0.007*"multilinguals" + 0.007*"spread" + 0.007*"dynamics" + 0.007*"belief" + 0.006*"media" + 0.006*"important" + 0.005*"social" + 0.005*"work"



In [200]:
long_vis_data22 = gensimvis.prepare(long_lda_model22, long_corpus22, long_dictionary22)
long_vis_data23 = gensimvis.prepare(long_lda_model22, long_corpus22, long_dictionary22)

In [201]:
pyLDAvis.display(long_vis_data22)

In [202]:
pyLDAvis.display(long_vis_data23)

In [203]:
# topics of the 10 shortest statements
short_statements22 = tokenized_words22.sort_values(by=['Word Count'],ascending=True)[:10]
short_lda_model22 = make_lda_model(short_statements22['Ethics'],3)

short_statements23 = tokenized_words23.sort_values(by=['Word Count'],ascending=True)[:10]
short_lda_model23 = make_lda_model(short_statements23['Ethics'],3)

In [204]:
short_vis_data22 = gensimvis.prepare(short_lda_model22, short_corpus22, short_dictionary22)
short_vis_data23 = gensimvis.prepare(short_lda_model22, short_corpus22, short_dictionary22)

In [206]:
pyLDAvis.display(short_vis_data22)

In [207]:
pyLDAvis.display(short_vis_data23)

## 5. LLM Evaluation <a name="sec5"></a>


In [17]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.environ['OPENAI_API_KEY']

In [20]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, 
                                 max_tokens=1000):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens,
    )
    return response.choices[0].message["content"]

In [23]:
delimiter = "####"
# add section for suggestions for improving
system_message = f"""
You are an assistant that grades papers. Each paper will be delimited \
by {delimiter} characters. Each paper will be assessed by how well it \
meets the points in the criteria list below. All criteria are weighted \
equally. You will assign each paper a score out of 100 based on how well \
it meets the criteria. Be very critical of each paper, it should be \
difficult to score a perfect 100. Additionally, for each paper write a brief \
summary of how they can improve their score i.e. which criteria the paper \
is lacking. Output a Python list of objects where \
each object has the following format: \
    'Ethics': <the original paper text>, \
    'Score': <the assigned score> ,\
    ‘Improvements’: <summary of how the paper can improve> \
Criteria:
The paper fully describes the rationale for the sample size used
The paper fully describes the study design, procedures, and materials to allow independent replication
The paper fully describes the measures of interest and their operationalizations 
The paper distinguishes explicitly between “confirmatory” (i.e., prespecified) and “exploratory” (i.e., not prespecified) analyses
The following have been made publicly available: The (processed) data, on which the analyses of the paper were based, all code and software (that is not copyright protected), all instructions, stimuli, and test materials (that are not copyright protected)
Includes a statement concerning the availability and location of all research items, including data, materials, and code relevant to your study
"""
user_message = f"""\
The data in this paper is derived from the Meta Ad Library. It contains publicly accessible ads run on Meta platforms by US politicians. Working with social media data carries risks of privacy issues and the right to be forgotten. However, our data analysis is limited to aggregated data presentations and only concerns ads published by public figures.
{delimiter}
This work follows the guidelines and the ethical considerations by Eysenbach and Till (2001); Moreno et al. (2013); Ramırez-Cifuentes et al. (2020). All the results provide aggregated estimates and do not include any information on individuals. The users in our study were fully aware of  the  public  nature  and  free  accessibility  of  the  content they posted since the subreddits are of public domain, are not password-protected, and have thousands of active subscribers.  Reddit’s  pseudonymous  accounts  make  the  retrieval  of  the  true  identity  of  users  unlikely.  Nevertheless, as  a  further  privacy  measure,  the  authors’  names  were anonymized before  using the data  for analysis.  Therefore, our research did not require informed consent.
{delimiter}
This study intersects with a number of topics related to the ethics of algorithmic platforms. For example, our analysis requires collecting data about which Tweets users view while using Twitter, and also requires human annotators to review and rate potentially abusive content. Overall, we agree with researchers who view this type of work as necessary for understanding and protecting democratic discourse (Fiske 2022), especially in terms of standard risk-benefit frameworks. Still, it is important to note different measures taken to address potential risks. While the holdback experiment is necessary, it is not ideal for many users to be excluded from algorithmic timeline features. Twitter has thus worked to provide more users access to algorithmic timelines while maintaining statistical robustness in the holdback experiment. As of 2020, the experiment included over 2 million active accounts in the reverse-chronological timeline group (Husz ́ar et al. 2022), but when this analysis was conducted in 2021, that number had been reduced to 630k. This study was not subject to an academic IRB process, however, it went through standard legal and privacy review processes at Twitter. Finally, the data used in this paper was fully anonymized for publication, following standard ethical procedures. We do not include any results that might disclose the identity of any account in the datasets.
{delimiter}
"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

[
  {
    'Ethics': 'The data in this paper is derived from the Meta Ad Library. It contains publicly accessible ads run on Meta platforms by US politicians. Working with social media data carries risks of privacy issues and the right to be forgotten. However, our data analysis is limited to aggregated data presentations and only concerns ads published by public figures.\n',
    'Score': 80,
    'Improvements': 'The paper should provide more details on the specific steps taken to ensure privacy and protect user identities. Additionally, it should address any potential biases in the data and discuss the limitations of using aggregated data for analysis.'
  },
  {
    'Ethics': 'This work follows the guidelines and the ethical considerations by Eysenbach and Till (2001); Moreno et al. (2013); Ramırez-Cifuentes et al. (2020). All the results provide aggregated estimates and do not include any information on individuals. The users in our study were fully aware of  the  public  nature  and 